In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]=""

In [2]:
import yaml
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf

import tensorflow as tf
from pathlib import Path

from tensorflow_tts.inference import AutoConfig
from tensorflow_tts.inference import TFAutoModel
from tensorflow_tts.inference import AutoProcessor

import IPython.display as ipd
from tqdm import tqdm

from textprocer.stress_modules import TextProcer
from pydub import AudioSegment
from pydub.effects import normalize


def get_normalized_text(text, max_text_length, max_sentence_length, lang, pc=0.0):
    """Return normalized and cleaned sentences"""
    sc = 1.0
    if lang == "en":
        sc = 1.0
    procers = {
        "ru": TextProcer(lang="ru"),
        "en": TextProcer(lang="en")
    }

    params = {
        'yo_chance': 1.0,
        'stress_chance': sc,
        'phoneme_chance': pc,
        'max_split_length': max_sentence_length,
        'expand_numbers': True,
        'to_lower': True,
        'sentences': [text],
    }

    result = procers[lang](**params)
    return " ".join(result['phoneme_sentences']).replace("*", "")

In [3]:
hifi_config = AutoConfig.from_pretrained('/home/newton/projects/TensorFlowTTS/examples/hifigan/conf/hifigan.v1.yaml')
hifi_gan = TFAutoModel.from_pretrained(
    config=hifi_config,
#     pretrained_path="/home/vera/projects/tts_v2/tts_2/TensorFlowTTS/multiband_melgan.v1_24k.yaml",
    pretrained_path="/home/newton/projects/TensorFlowTTS/examples/hifigan/exp/train.hifigan.v1/checkpoints/generator-960000.h5",
)


2022-02-03 09:10:50.250178: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-02-03 09:10:50.250233: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: newton
2022-02-03 09:10:50.250241: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: newton
2022-02-03 09:10:50.250435: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 495.29.5
2022-02-03 09:10:50.250467: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 495.29.5
2022-02-03 09:10:50.250475: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 495.29.5
2022-02-03 09:10:50.250808: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in perf

In [189]:
from tensorflow_tts.processor import RuslanProcessor
from tensorflow_tts.processor.ruslan import RUSLAN_SYMBOLS


processor = RuslanProcessor(
    "/home/newton/datasets/new_ruslan/ruslan_data",
    symbols=RUSLAN_SYMBOLS,
    cleaner_names="basic_cleaners",
)


config = AutoConfig.from_pretrained("../examples/fastspeech2/conf/fastspeech2.v1.yaml")


fast_speech2 = TFAutoModel.from_pretrained(
     config=config,
#      pretrained_path="./thorsten-tacotron2.h5"
#      pretrained_path="/home/newton/checkpoints/fs_sobchak/checkpoints/model-150000.h5",
#      pretrained_path="/home/newton/checkpoints/fs_sobchak/checkpoints/model-40000.h5",
     pretrained_path="/home/newton/checkpoints/fs_sobchak/checkpoints/model-15000.h5",
     enable_tflite_convertible=True
#      pretrained_path="/home/newton/checkpoints/fs_narcos/checkpoints/model-10000.h5"
#      pretrained_path="/home/newton/checkpoints/fs_taco/checkpoints/model-150000.h5"
)

In [190]:
# text = 'добрый день дорогие дамы и господа.'
# text = 'д+обрый день дорог+ие д+амы и господ+а.'
# text = '+игорь ск+отников гей.'
# text = 'прив+ет, дорогой ив+анов ив+ан ив+анович. я хоч+у подздравить теб+я с днем рожд+ения. жел+аю теб+е сч+астья, здор+овья и карьерного роста, к+ак у мен+я.'
# text = 'дорогой ив+анов ив+ан ив+анович, мы тут встр+етились с в+ашим знак+омым, и он попрос+ил мен+я л+ично поздр+авить вас с н+овым г+одом! к+ак вы зн+аете, моб+ильного телеф+она у мен+я нет, по+этому отправл+яю вам голосов+ое сообщ+ение!'

text = get_normalized_text("Мокер попытался выровнить направл+ение бес+еды.", 10000, 100, "ru")
# text = get_normalized_text("Шансы семь из десяти.", 10000, 100, "ru")
# text = get_normalized_text("Он был похож на страшно истаскавшегося юношу.", 10000, 100, "ru")
# text = get_normalized_text("С тринадцати лет Мар+усю окружали р+азвитые интеллигентные хорошо воспитанные юноши.", 10000, 100, "ru")
# text = get_normalized_text("Начиналась моя жизнь в америки крайне безмет+ежно", 10000, 100, "ru")
# text = get_normalized_text("В пакете что то щ+ёлкало свистело и цар+апалось, при этом доносились тягостные вздохи", 10000, 100, "ru")
# text = get_normalized_text("Все усилия чтобы это сделать, даже ездили в санкт петербург..", 10000, 100, "ru")
# text = get_normalized_text("Чтобы один предмет не пропустить больше чем два раза", 10000, 100, "ru")
# text = get_normalized_text("Мы разучивали вместе эти песни на английском с трёх лет.", 10000, 100, "ru")
# text = get_normalized_text("А в театре очень много нюансов и очень много мелочей.", 10000, 100, "ru")
# text = get_normalized_text("Мама у меня была категорически не согласна но я в параллели.", 10000, 100, "ru")
# text = get_normalized_text("Я очень сильно её боялась, я маме говорю, мам пожалуста давай пере+едем.", 10000, 100, "ru")
# text = get_normalized_text("Ну я же извинилась, пидар+аз ты клятый .", 10000, 100, "ru")


# input_ids = processor.text_to_sequence(cleaners.german_cleaners(text))
input_ids = processor.text_to_sequence(text)


In [191]:
def tts(input_ids):
    mel_before, mel_after, duration_outputs, _, _ = fast_speech2.inference(
        input_ids=tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0),
        speaker_ids=tf.convert_to_tensor([0], dtype=tf.int32),
        speed_ratios=tf.convert_to_tensor([1.0], dtype=tf.float32),
        f0_ratios=tf.convert_to_tensor([1.0], dtype=tf.float32),
        energy_ratios=tf.convert_to_tensor([1.0], dtype=tf.float32)
    )
    return hifi_gan.inference(mel_after)[0, :, 0]

In [192]:
ipd.Audio(tts(input_ids), rate=22050)